# Workflow

1. Identify datasets to be predicted
2. Parse the datasets into dataframes
3. Perform pre-processing on dataframes
4. Train chosen model on full training data
5. Predict labels for all datasets
6. Save predictions into CSV files

# Identify datasets to be predicted

In [1]:
## libraries to read and parse json file
import json
import pandas as pd
import os
import sys

In [2]:
## get current working directory
os.getcwd()

'/Users/claudia/DSA4262-ACMXZ/prediction'

# Parse datasets into dataframes

### Functions needed to parse json files

In [3]:
## function to get key of a dictionary
def get_key(dictionary):
    key_object = dictionary.keys()
    key = list(key_object)[0]
    return key

In [4]:
## function to help concatenate columns to get transcript, position, nucleotides
def concat_col(transcript, position, nucleotide, n):
    t_df = pd.DataFrame([transcript]*n)
    p_df = pd.DataFrame([position]*n)
    nu_df = pd.DataFrame([nucleotide]*n)
    n_df = pd.DataFrame([n]*n)

    ## concat columns together
    final_df = pd.concat([t_df, p_df, nu_df, n_df], axis = 1)
    final_df.columns = ['transcript', 'position', 'nucleotides', 'reads_count']
    return final_df

In [5]:
## function to parse line in json file
def parse_line(line):
    ## get transcript
    t = get_key(line)

    ## get position
    p = get_key(line[t])

    ## get nucleotide seq
    nu = get_key(line[t][p])

    ## get number of reads
    reads_count = len(line[t][p][nu])

    ## get dataframe of list of reads
    reads = pd.DataFrame(line[t][p][nu])

    ## concat columns together to get transcript, position, nucleotides and all dwelling time, std, mean
    df = pd.concat([concat_col(t, p, nu, reads_count), reads], axis = 1)
    df.columns = ['transcript', 'position', 'nucleotides', 'reads_count', 'dwellingtime_-1', 'std_-1', 'mean_-1', 'dwellingtime_0', 'std_0', 'mean_0', 'dwellingtime_+1', 'std_+1', 'mean_+1']

    return df

In [19]:
## function to breakdown dataframe into smaller sizes and save it
def save_file(df, filename, nrows = 2500000):
    total_rows = len(df)
    start, stop, count = 0, nrows, 1

    ## use while loop to break the dataframe into smaller dataframes
    while stop < total_rows:
        print(start, stop)
        temp_df = pd.DataFrame(df.iloc[start:stop, :])
        fname = f"{filename}_{count}.parquet"
        temp_df.to_parquet(fname)
        print(f"Saved a file called {fname}")
        count += 1
        start += nrows
        stop += nrows
    
    stop = total_rows
    print(start, stop)
    temp_df = pd.DataFrame(df.iloc[start:stop, :])
    fname = f"{filename}_{count}.parquet"
    temp_df.to_parquet(fname)
    print(f"Saved a file called {fname}")

### Parse datasets

In [11]:
def parse(file, filename):
    ## open file
    data = [json.loads(line) for line in open(file, 'r')]

    ## parse all lines into dataframes
    reads = [parse_line(data[i]) for i in range(len(data))]

    ## concatenate dataframes
    result_df = pd.concat(reads, axis = 0)
    print(f"Shape of Dataset = {result_df.shape}")

    ## save dataframe into parquet files
    save_file(result_df, filename)

    return result_df

In [12]:
dataset1_path = "/Users/claudia/Downloads/dataset1.json"
dataset1_filename = "../data/final_round/dataset1"
dataset1 = parse(dataset1_path, dataset1_filename)

Shape of Dataset = (7907952, 13)
0 5000000
Saved a file called ../data/final_round/dataset1_1.parquet
5000000 7907952
Saved a file called ../data/final_round/dataset1_2.parquet


In [22]:
dataset2_path = "/Users/claudia/Downloads/dataset2.json"
dataset2_filename = "../data/final_round/dataset2"
dataset2 = parse(dataset2_path, dataset2_filename)

Shape of Dataset = (6903936, 13)
0 2500000
Saved a file called ../data/final_round/dataset2_1.parquet
2500000 5000000
Saved a file called ../data/final_round/dataset2_2.parquet
5000000 6903936
Saved a file called ../data/final_round/dataset2_3.parquet


In [23]:
dataset3_path = "/Users/claudia/Downloads/dataset3.json"
dataset3_filename = "../data/final_round/dataset3"
dataset3 = parse(dataset3_path, dataset3_filename)

Shape of Dataset = (1171940, 13)
0 1171940
Saved a file called ../data/final_round/dataset3_1.parquet


### Read parsed datasets

In [3]:
os.chdir("../data/final_round/")
os.getcwd()

'/Users/claudia/DSA4262-ACMXZ/data/final_round'

In [4]:
dataset1_dfs = [pd.read_parquet(file) for file in os.listdir() if file.startswith("dataset1")]
dataset1 = pd.concat(dataset1_dfs)
dataset1.shape

(7907952, 13)

In [5]:
dataset2_dfs = [pd.read_parquet(file) for file in os.listdir() if file.startswith("dataset2")]
dataset2 = pd.concat(dataset2_dfs)
dataset2.shape

(6903936, 13)

In [6]:
dataset3_dfs = [pd.read_parquet(file) for file in os.listdir() if file.startswith("dataset3")]
dataset3 = pd.concat(dataset3_dfs)
dataset3.shape

(1171940, 13)

# Train model on full training dataset

In [7]:
from sklearn.ensemble import RandomForestClassifier

In [9]:
## load in Train Set
X_train_path = "../raw_data/X_raw_enc.parquet"
X_train = pd.read_parquet(X_train_path)
y_train_path = "../raw_data/y_raw.parquet"
y_train = pd.read_parquet(y_train_path)

### convert y_train into int
y_train = y_train.values.ravel()
y_train = y_train.astype(int)

In [10]:
rfe_features = ['std_-1_25', 'std_-1_50', 'std_-1_75', 'std_-1_mean', 'std_-1_min',
       'mean_-1_25', 'mean_-1_50', 'mean_-1_75', 'mean_-1_mean', 'mean_-1_min',
       'dwelling_time_0_50', 'dwelling_time_0_mean', 'std_0_25', 'std_0_50',
       'std_0_75', 'std_0_mean', 'std_0_min', 'std_0_max', 'mean_0_25',
       'mean_0_50', 'mean_0_75', 'mean_0_mean', 'mean_0_min', 'mean_0_max',
       'dwelling_time_+1_mean', 'std_+1_25', 'std_+1_50', 'mean_+1_25',
       'mean_+1_50', 'mean_+1_75', 'mean_+1_mean', 'mean_+1_min',
       'mean_+1_max', 'relative_position', 'position_1_G', 'position_5_T']

In [11]:
rfc = RandomForestClassifier(random_state = 42, n_estimators = 220, max_features = "sqrt", max_depth = 30,
                        min_samples_split = 2, min_samples_leaf = 1, bootstrap = False)

rfc.fit(X_train[rfe_features], y_train)

RandomForestClassifier(bootstrap=False, max_depth=30, n_estimators=220,
                       random_state=42)

# Perform pre-processing on dataframes

### Functions needed for pre-processing

In [12]:
import numpy as np
import pickle
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from category_encoders import OneHotEncoder

sys.path.append(os.path.abspath("../../util/model"))
from training import get_percent

In [13]:
# gene_id and labels removed from groupby and temp.columns
def feature_eng(df):
    temp = pd.DataFrame(df.groupby(['transcript', 'position', 'nucleotides', 'reads_count'], as_index=False)
                           .agg({'dwellingtime_-1': [get_percent(25), get_percent(50), get_percent(75), np.mean, np.min, np.max],
                                'std_-1': [get_percent(25), get_percent(50), get_percent(75), np.mean, np.min, np.max],
                                'mean_-1': [get_percent(25), get_percent(50), get_percent(75), np.mean, np.min, np.max],
                                'dwellingtime_0': [get_percent(25), get_percent(50), get_percent(75), np.mean, np.min, np.max],
                                'std_0': [get_percent(25), get_percent(50), get_percent(75), np.mean, np.min, np.max],
                                'mean_0': [get_percent(25), get_percent(50), get_percent(75), np.mean, np.min, np.max],
                                'dwellingtime_+1': [get_percent(25), get_percent(50), get_percent(75), np.mean, np.min, np.max],
                                'std_+1': [get_percent(25), get_percent(50), get_percent(75), np.mean, np.min, np.max],
                                'mean_+1': [get_percent(25), get_percent(50), get_percent(75), np.mean, np.min, np.max]}))
    temp.columns = ['transcript', 'position', 'nucleotides', 'reads_count',
                        'dwelling_time_-1_25', 'dwelling_time_-1_50', 'dwelling_time_-1_75', 'dwelling_time_-1_mean','dwelling_time_-1_min', 'dwelling_time_-1_max',
                        'std_-1_25', 'std_-1_50', 'std_-1_75', 'std_-1_mean','std_-1_min', 'std_-1_max',
                        'mean_-1_25', 'mean_-1_50', 'mean_-1_75', 'mean_-1_mean','mean_-1_min', 'mean_-1_max',
                        'dwelling_time_0_25', 'dwelling_time_0_50', 'dwelling_time_0_75', 'dwelling_time_0_mean','dwelling_time_0_min','dwelling_time_0_max',
                        'std_0_25', 'std_0_50', 'std_0_75', 'std_0_mean','std_0_min', 'std_0_max',
                        'mean_0_25', 'mean_0_50', 'mean_0_75', 'mean_0_mean','mean_0_min', 'mean_0_max',
                        'dwelling_time_+1_25', 'dwelling_time_+1_50', 'dwelling_time_+1_75', 'dwelling_time_+1_mean','dwelling_time_+1_min','dwelling_time_+1_max',
                        'std_+1_25', 'std_+1_50', 'std_+1_75', 'std_+1_mean','std_+1_min', 'std_+1_max',
                        'mean_+1_25', 'mean_+1_50', 'mean_+1_75', 'mean_+1_mean','mean_+1_min', 'mean_+1_max']
    return temp

In [14]:
# gene_id removed from groupby
def relative_position(df):
    df["position"] = df["position"].astype(int)

    ## find relative position of each read in each transcript
    df["relative_position"] = df.groupby(["transcript"])["position"].transform(lambda x: (x - x.min())/(x.max()-x.min()))

    ## note: have NAs because there's transcripts with only one position
    ## fill the NAs with 0
    df["relative_position"] = df["relative_position"].fillna(0)

    return df

In [15]:
## variables needed for encoding
pipe = pickle.load(open("../raw_data/encoding_pipeline.pkl", "rb"))

cols_to_map = ['reads_count', 'dwelling_time_-1_25', 'dwelling_time_-1_50', 'dwelling_time_-1_75', 
                'dwelling_time_-1_mean', 'dwelling_time_-1_min', 'dwelling_time_-1_max', 'std_-1_25', 
                'std_-1_50', 'std_-1_75', 'std_-1_mean', 'std_-1_min', 'std_-1_max', 'mean_-1_25', 
                'mean_-1_50', 'mean_-1_75', 'mean_-1_mean', 'mean_-1_min', 'mean_-1_max', 
                'dwelling_time_0_25', 'dwelling_time_0_50', 'dwelling_time_0_75', 'dwelling_time_0_mean', 
                'dwelling_time_0_min', 'dwelling_time_0_max', 'std_0_25', 'std_0_50', 'std_0_75', 
                'std_0_mean', 'std_0_min', 'std_0_max', 'mean_0_25', 'mean_0_50', 'mean_0_75', 'mean_0_mean', 
                'mean_0_min', 'mean_0_max', 'dwelling_time_+1_25', 'dwelling_time_+1_50', 'dwelling_time_+1_75', 
                'dwelling_time_+1_mean', 'dwelling_time_+1_min', 'dwelling_time_+1_max', 'std_+1_25', 
                'std_+1_50', 'std_+1_75', 'std_+1_mean', 'std_+1_min', 'std_+1_max', 'mean_+1_25', 'mean_+1_50', 
                'mean_+1_75', 'mean_+1_mean', 'mean_+1_min', 'mean_+1_max', 'relative_position', 'position_0_C', 
                'position_0_G', 'position_0_T', 'position_0_A', 'position_1_A', 'position_1_G', 'position_1_T', 
                'position_2_A', 'position_2_G', 'position_3_A', 'position_4_C', 'position_5_C', 'position_5_A', 
                'position_5_T', 'position_6_T', 'position_6_A', 'position_6_G', 'position_6_C']

In [16]:
def encoding(df, columns_to_map):
    id_val = df[['transcript','position']] ## needed to concat with pred proba for submission

    for i in range(7):
        df['position_' + str(i)] = df['nucleotides'].apply(lambda x: x[i])
    
    df_enc = pd.DataFrame({col: vals for vals, col in zip(pipe.transform(df).T, columns_to_map)})

    return df_enc, id_val

### Perform pre-processing on dataframes

In [17]:
## preprocess dataset 1
percentile_df1 = feature_eng(dataset1)
print(percentile_df1.shape)

relative_position_df1 = relative_position(percentile_df1)
print(relative_position_df1.shape)

encoded_df1, id_val_df1 = encoding(relative_position_df1, cols_to_map)
print(encoded_df1.shape, id_val_df1.shape)

data1_pp = encoded_df1

(90810, 58)
(90810, 59)
(90810, 74) (90810, 2)


In [18]:
## preprocess dataset 2
percentile_df2 = feature_eng(dataset2)
print(percentile_df2.shape)

relative_position_df2 = relative_position(percentile_df2)
print(relative_position_df2.shape)

encoded_df2, id_val_df2 = encoding(relative_position_df2, cols_to_map)
print(encoded_df2.shape, id_val_df2.shape)

data2_pp = encoded_df2

In [ ]:
## preprocess dataset 3
percentile_df3 = feature_eng(dataset3)
print(percentile_df3.shape)

relative_position_df3 = relative_position(percentile_df3)
print(relative_position_df3.shape)

encoded_df3, id_val_df3 = encoding(relative_position_df3, cols_to_map)
print(encoded_df3.shape, id_val_df3.shape)

data3_pp = encoded_df3

# Predict using random forest classifier

In [ ]:
## predict data1_pp
data1_pred = rfc.predict_proba(data1_pp[rfe_features])[:,1]
print(len(data1_pred))

## predict data2_pp
data2_pred = rfc.predict_proba(data2_pp[rfe_features])[:,1]
print(len(data2_pred))

## predict data3_pp
data3_pred = rfc.predict_proba(data3_pp[rfe_features])[:,1]
print(len(data3_pred))

In [ ]:
## convert predicted data into dataframes
data1_res = pd.DataFrame(data1_pred, columns = ['score'])
data2_res = pd.DataFrame(data2_pred, columns = ['score'])
data3_res = pd.DataFrame(data3_pred, columns = ['score'])

In [ ]:
## concat predicted probabilities with id_val columns
data1_res = pd.concat([id_val_df1, data1_res], axis = 1)
print(data1_res.shape)

data2_res = pd.concat([id_val_df2, data2_res], axis = 1)
print(data2_res.shape)

data3_res = pd.concat([id_val_df3, data3_res], axis = 1)
print(data3_res.shape)

In [ ]:
## rename columns for submission
data1_res.rename(columns = {'transcript': 'transcript_id', 'position': 'transcript_position'})
data2_res.rename(columns = {'transcript': 'transcript_id', 'position': 'transcript_position'})
data3_res.rename(columns = {'transcript': 'transcript_id', 'position': 'transcript_position'})

In [ ]:
data1_res.head()

In [ ]:
data2_res.head()

In [ ]:
data3_res.head()

In [ ]:
## save files as csv
data1_res.to_csv("acmxz_dataset1.csv")
data2_res.to_csv("acmxz_dataset2.csv")
data3_res.to_csv("acmxz_dataset3.csv")